In [58]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 5, algorithm='brute') # Instancier le modele de Machine Learning
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance 
import pandas as pd
import numpy as np

In [59]:
paris_listings = pd.read_csv("paris_airbnb.csv")

In [60]:
paris_listings.head(4)

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews
0,100%,NaN,1.0,48.83349,2.31852,Paris,75014,Île-de-France,2,Entire home/apt,0.0,1.0,0.0,$75.00,$50.00,$0.00,3,15,8
1,100%,NaN,1.0,48.85100,2.35869,Paris,75004,Ile-de-France,2,Entire home/apt,0.0,1.0,1.0,$115.00,$36.00,$0.00,1,90,173
2,100%,NaN,2.0,48.85758,2.35275,Paris,75004,Île-de-France,4,Entire home/apt,2.0,1.0,2.0,$115.00,$50.00,$200.00,10,23,243
3,100%,NaN,1.0,48.86528,2.39326,Paris,75020,Ile-de-France,3,Entire home/apt,1.0,1.0,1.0,$90.00,NaN,NaN,3,365,1


****Exemple: Création une instance de la classe KNeighborsRegressor. Voir l'énoné sur les diapos.**** 
- n_neighnors: 5
- algorithm: brut

In [61]:
paris_listings["price"]=paris_listings["price"].apply(lambda x: x.replace(",", ""))
paris_listings["price"]=paris_listings["price"].apply(lambda x: x.replace("$", ""))
paris_listings["price"]=paris_listings["price"].apply(lambda x: float(x))

In [62]:
paris_listings.head(4)

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews
0,100%,NaN,1.0,48.83349,2.31852,Paris,75014,Île-de-France,2,Entire home/apt,0.0,1.0,0.0,75.0,$50.00,$0.00,3,15,8
1,100%,NaN,1.0,48.85100,2.35869,Paris,75004,Ile-de-France,2,Entire home/apt,0.0,1.0,1.0,115.0,$36.00,$0.00,1,90,173
2,100%,NaN,2.0,48.85758,2.35275,Paris,75004,Île-de-France,4,Entire home/apt,2.0,1.0,2.0,115.0,$50.00,$200.00,10,23,243
3,100%,NaN,1.0,48.86528,2.39326,Paris,75020,Ile-de-France,3,Entire home/apt,1.0,1.0,1.0,90.0,NaN,NaN,3,365,1


In [63]:
drop_columns = ['room_type','city', 'state', 'latitude', 'longitude', 'zipcode', 'host_response_rate', 'host_acceptance_rate','host_listings_count']

In [64]:
paris_listings = paris_listings.drop(drop_columns, axis=1)
print(paris_listings.isnull().sum())

accommodates            0
bedrooms               24
bathrooms              58
beds                   14
price                   0
cleaning_fee         1750
security_deposit     1680
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64


In [65]:
paris_listings = paris_listings.drop(['security_deposit', 'cleaning_fee'], axis=1) # ça supp les colonnes
paris_listings = paris_listings.dropna(axis=0) # ça va supp toutes les lignes qui contiennet des valeurs manquantes
print(paris_listings.isnull().sum())

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64


In [66]:
# 1) soustraire chaque valeur dans la colonne par moyenne
first_transformation = paris_listings['maximum_nights'] - paris_listings['maximum_nights'].mean()

# 2) Diviser chaque valeur dans la colonne par l'ecart type
normalized_col = first_transformation / first_transformation.std()

In [67]:
normalized_listings = (paris_listings - paris_listings.mean()) / paris_listings.std()
normalized_listings['price'] = paris_listings['price']
print('Voilà nos valeurs normalisées selon la distribution normale standard pour les 3 1ère lignes')
print()
print(normalized_listings.head(3))
print()
print('On remarque sur la colonne price, on a nos valeurs non normalisées')

Voilà nos valeurs normalisées selon la distribution normale standard pour les 3 1ère lignes

   accommodates  bedrooms  bathrooms     beds  price  minimum_nights  \
0     -0.767566 -1.486372  -0.293537 -1.49583   75.0       -0.156795   
1     -0.767566 -1.486372  -0.293537 -0.64526  115.0       -0.212407   
2      0.503868  0.892605  -0.293537  0.20531  115.0        0.037847   

   maximum_nights  number_of_reviews  
0       -0.979751          -0.535593  
1       -0.841737           1.852943  
2       -0.965029           2.866261  

On remarque sur la colonne price, on a nos valeurs non normalisées


In [68]:
# 1)Diviser le dataset en deux partie
train_df = normalized_listings.iloc[0:6000]
test_df = normalized_listings.iloc[6000:]

In [69]:
train_columns = ['accommodates', 'bedrooms']

In [70]:
# Ensuite il faut adapter le modele aux données: C'est la méthode fit
knn.fit(train_df[train_columns], train_df['price'])

KNeighborsRegressor(algorithm='brute')

In [71]:
# Enfin on va utiliser le modèle pour faire de prédiction
predictions = knn.predict(test_df[train_columns])

In [72]:
print(predictions)

[137.6  74.6  74.6 ...  97.8  74.6 103.8]


### Calcul l'erreur quadratique moyenne

In [73]:
two_features_mse = mean_squared_error(test_df['price'], predictions)
two_features_rmse = two_features_mse**(1/2)
print(two_features_mse)
print(two_features_rmse)

8495.167803860199
92.16923458432427


### Test du modèle sur les autres colonnes: Ici on entaine le nombre de caractéristique que l'ont souhaite.

In [74]:
features = ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']
knn.fit(train_df[features], train_df['price'])
four_prediction = knn.predict(test_df[features])

four_mse = mean_squared_error(test_df['price'], four_prediction)
four_rmse = four_mse**(1/2)

print(four_mse)
print(four_rmse)

5966.076515388628
77.24038137780411


### Utilisation de toutes les caractéristiques: ici on entraine toutes les autres caractéristiques. Utilisons toutes les colonnes sauf 'price'

In [76]:
features = train_df.columns.to_list()
features.remove('price')

knn.fit(train_df[features], train_df['price'])
all_features_predictions = knn.predict(test_df[features])

all_features_mse = mean_squared_error(test_df['price'], all_features_predictions)
all_features_rmse = all_features_mse**(1/2)

print(all_features_mse)
print(all_features_rmse)


11697.899634846115
108.15682888678882


On constate que cette valeur de RMSE a augmenté par auprécédent modèle. 
lorsque nous avons utilisé seulement 4 caractéristiques du modèles, 'accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews', on obtient un meilleur modèle. Parcontre, en utilisant toutes les caractéristiques, notre modèle a perdu en qualité. Cela signifie la selection des caractéristiques pertinantes est importante et que l'utilisation des caractéristiques supplémentaires n'améliore pas le modèle, cad, la performance de la prédiction.